In [0]:
sales_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/parveen.r@live.com/sales.csv")

In [0]:
# Show sample data
print("Sample Sales Data:")
sales_df.show(5)

In [0]:
display(sales_df.limit(10))

In [0]:
# Write DataFrame as Delta Table (managed)
sales_df.write.format("delta").mode("overwrite").saveAsTable("sales_delta_managed")
print("Delta table 'sales_delta_managed' created successfully.")

In [0]:
# Verify by reading table
sales_table_df = spark.table("sales_delta_managed")
sales_table_df.printSchema()
display(sales_table_df.limit(10))

In [0]:
# Write DataFrame as external Delta table
external_path = "/dbfs/delta/sales_external_delta"
sales_df.write.format("delta").mode("overwrite").save(external_path)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS sales_delta_external
USING DELTA
LOCATION '{external_path}'
""")
print("External Delta table 'sales_delta_external' created successfully.")

In [0]:
# Show contents of external table
spark.table("sales_delta_external").show(5)

In [0]:
%sql
UPDATE sales_delta_external
SET UnitPrice = UnitPrice + 1
WHERE SalesOrderNumber = 'SO1000';


In [0]:
sales_log = spark.read.format("json").load("dbfs:/dbfs/delta/sales_external_delta/_delta_log/00000000000000000000.json")
display(sales_log)

In [0]:
%sql
DESCRIBE HISTORY sales_delta_external


In [0]:
%sql
select * from sales_delta_external

In [0]:
%sql
SELECT * FROM sales_delta_external VERSION AS OF 9;


In [0]:
%sql
SELECT * FROM sales_delta_external VERSION AS OF 1

In [0]:
%sql
RESTORE TABLE sales_delta_external TO VERSION AS OF 7

In [0]:
%sql
select * from sales_delta_external

In [0]:
# Disable retention duration safety check
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

In [0]:
%sql
VACUUM sales_delta_external RETAIN 0 HOURS